In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
from datetime import datetime
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR')

'fr_FR'

In [2]:
#Sur Strava, nous pouvons récupérer le département de l'activité mais pas la région. Nous avons besoin de ces deux fichiers pour lier le département à une région
import json
with open('regions.json', 'r') as fichier_regions:
    regions = json.load(fichier_regions)
with open('departments.json', 'r') as fichier_departments:
    departements = json.load(fichier_departments)

In [69]:
df = pd.DataFrame(columns = ['Athlète', 'Date', 'Région', 'Département', 'Distance', 'Durée', 'Dénivelé', 'Puissance moyenne', 'Effort', 'Charge entraînement', 'Intensité', 'Vitesse moyenne', 'Vitesse max', 'Météo', 'Température', 'Humidité', 'Température ressentie', 'Vitesse vent', 'Direction vent'])

In [85]:
#Lancement du driver
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

driver.get("https://www.strava.com/login")

In [86]:
#Connection à Strava
driver.find_element("id", "email").send_keys('df119a65719856@crankymonkey.info')
driver.find_element("id", "password").send_keys('Selenium159')
driver.find_element("id", "login-button").click()

In [6]:
#Fonction pour lier un département à une région
def trouver_region_par_departement(phrase):
    for numero_departement, nom_departement in departements.items():
        if nom_departement.lower() in phrase.lower():
            for region, numeros_departements in regions.items():
                if numero_departement in numeros_departements:
                    return region, nom_departement
    return None

In [7]:
#Fonction pour rechercher un nouvel athlète
def nouvel_athlete(nom):
    driver.find_element("id", "open-global-search-button").click()
    #Nous recherchons les athlètes via la barre de recherche
    driver.find_element("id", "global-search-field").send_keys(nom)
    time.sleep(3)
    driver.find_element("id", "global-search-field").send_keys(Keys.RETURN)
    #clique sur le premier élément retourné par la recherche
    time.sleep(3)
    driver.find_element(By.CLASS_NAME, "athlete-name-link").click()

In [11]:
#Fonction pour récupérer l'ensemble des activités Vélo entre 2021 et 2023 compris. Retourne une liste de lien d'activités
def activites_athlete():
    activities = []
    for year in [None,"24 janv. 2022 - 23 janv. 2023", "25 janv. 2021 - 24 janv. 2022"]:
        if(year!=None):
            dropdown_menu = driver.find_element(By.XPATH,"//div[@class='drop-down-menu drop-down-sm enabled']").click()
            desired_text = year
            print(desired_text)
            desired_element = driver.find_element(By.XPATH,f"//a[text()='{desired_text}']")
            desired_element.click()
        
        time.sleep(10)
        driver.find_element(By.PARTIAL_LINK_TEXT, "Par mois").click()
        time.sleep(10)

        for month in range(1,13):
            if month < 10:
                month = '0' + str(month)
            if (year == "24 janv. 2022 - 23 janv. 2023"):
                id_mois = 'interval-2022' + str(month)
            elif (year == "25 janv. 2021 - 24 janv. 2022"):
                id_mois = 'interval-2021' + str(month)
            else:
                id_mois = 'interval-2023' + str(month)
            print(id_mois)
            time.sleep(10)
            driver.find_element("id", id_mois).click()

            time.sleep(40)

            elements = driver.find_elements("id", "feed-entry-null")
            while(True):
                try:
                    elements[0].find_elements("tag name", "a")
                except Exception:
                    print('Exception')
                    time.sleep(1)
                    elements = driver.find_elements("id", "feed-entry-null")
                break
            
            for element in elements:
                links = element.find_elements("tag name", "a")          
                try:
                    sport = element.find_element("tag name", "title").text
                except Exception:
                    continue
                if(sport!="Vélo"):
                    continue            
                filtered_links = [link.get_attribute("href") for link in links if link.get_attribute("href").startswith("https://www.strava.com/activities/")]

                activities.append(filtered_links[0])

            print(activities)
            time.sleep(3)
    return activities

In [66]:
#Fonction qui ajoute dans les activités dans le dataframe
def conversion_ajout_dataframe(name_athlete, lien):
    driver.get(lien)
    details = driver.find_element(By.CLASS_NAME, 'details')
    array_date = details.text.split(' ')[:4]
    location_element = details.find_element(By.CSS_SELECTOR, 'span.location').text
    region, departement = trouver_region_par_departement(location_element)
    format_date = '%A %d %B %Y'
    date_string = ' '.join(array_date)
    ma_date = datetime.strptime(date_string, format_date)
    div = driver.find_element(By.CLASS_NAME, 'spans8.activity-stats.mt-md.mb-md')
    lignes = div.text.split('\n')

    activity = []
    for i in range(0,len(lignes),2):
        activity.append(lignes[i])

    for element in lignes:
        if "Mesure d'effort" in element:
            activity.pop(3) #pour éviter la mesure d'effort
    ind_traitement_simple = [2, 3, 4, 5, 6, 10, 11, 12]
    depense_energie = False
    for element in lignes:
        if 'kJ' in element:
            depense_energie = True
    if depense_energie==False:
        activity.insert(3, '999 W') #pour ajouter une dépense d'énergie fictive
        activity.insert(4, '9 999 kJ')
    if "Charge d’entraînement" not in lignes:
        activity.insert(5, '999') #pour ajouter une charge d'entrainement fictive
        activity.insert(6, '99%')
    for i in range(len(activity)):
        if i in ind_traitement_simple:
            activity[i] = int(''.join(c for c in activity[i] if c.isdigit()))
        elif i in [0, 13]:
            activity[i] = float(re.search(r'\d+,\d+', activity[i]).group().replace(',', '.'))

    resultats = re.findall(r'\b\d+,\d+\b', activity[8])
    nombres = [float(nombre.replace(',', '.')) for nombre in resultats]
    del activity[7]
    activity.insert(7, nombres[1])
    activity.insert(7, nombres[0])
    del activity[9]    
    activity.insert(0, name_athlete)
    activity.insert(1, departement)
    activity.insert(1, region)
    activity.insert(1, ma_date)
    if(region):
        df.loc[df.shape[0]] = activity
    else:
        print("Pas de région trouvée pour l'activité:" + str(lien))

In [25]:
#Bloc principal appelant les fonctions pour avoir un tableau de toutes les activités des athlètes
noms_athletes = ['David Gaudu']
athletes_and_activities = []
for name in noms_athletes:
    nouvel_athlete(name)
    athletes_and_activities.append({name:activites_athlete()})

In [83]:
athletes_and_activities

[{'David Gaudu': ['https://www.strava.com/activities/6708055407',
   'https://www.strava.com/activities/6666600163',
   'https://www.strava.com/activities/6645025133',
   'https://www.strava.com/activities/6630107044',
   'https://www.strava.com/activities/6626735565',
   'https://www.strava.com/activities/6620455980',
   'https://www.strava.com/activities/6911189266',
   'https://www.strava.com/activities/6902044653',
   'https://www.strava.com/activities/6897140549',
   'https://www.strava.com/activities/6886455871',
   'https://www.strava.com/activities/7101646216',
   'https://www.strava.com/activities/7101643458',
   'https://www.strava.com/activities/7101641747',
   'https://www.strava.com/activities/7101639796',
   'https://www.strava.com/activities/7101639619',
   'https://www.strava.com/activities/7101639269',
   'https://www.strava.com/activities/7101638869',
   'https://www.strava.com/activities/7101638980',
   'https://www.strava.com/activities/7101638524',
   'https://www.

In [87]:
#Une fois que l'on a les activités des athlètes, la boucle les parcourt et appelle la fonction ajoutant ces détails dans le dataframe
for athlete in athletes_and_activities[0].keys():
    for activity in athletes_and_activities[0][athlete]:
        try:
            conversion_ajout_dataframe(athlete, activity)
        except Exception:
            print('Exception: ' + str(activity))
        time.sleep(5)
df

Exception: https://www.strava.com/activities/6708055407
Exception: https://www.strava.com/activities/6630107044
Exception: https://www.strava.com/activities/6626735565
Exception: https://www.strava.com/activities/6620455980
Exception: https://www.strava.com/activities/7101621181
Exception: https://www.strava.com/activities/7101674645
Exception: https://www.strava.com/activities/7101672819
Exception: https://www.strava.com/activities/7101669949
Exception: https://www.strava.com/activities/7101667760
Exception: https://www.strava.com/activities/7101664690
Exception: https://www.strava.com/activities/6931530798
Exception: https://www.strava.com/activities/7201081432
Exception: https://www.strava.com/activities/7199087194
Exception: https://www.strava.com/activities/7193824797
Exception: https://www.strava.com/activities/7185674137
Exception: https://www.strava.com/activities/7179150137
Exception: https://www.strava.com/activities/7168980179
Exception: https://www.strava.com/activities/716

,Athlète,Date,Région,Département,Distance,Durée,Dénivelé,Puissance moyenne,Effort,Charge entraînement,Intensité,Vitesse moyenne,Vitesse max,Météo,Température,Humidité,Température ressentie,Vitesse vent,Direction vent
0,David Gaudu,2023-01-13,Auvergne-Rhône-Alpes,Ain,90.87,2:58:12,1268,999,9999,999,99,30.6,66.9,Quelques nuages,8,88,4,23.7,SSW
1,David Gaudu,2023-01-11,Bretagne,Finistère,91.30,3:02:50,1135,999,9999,999,99,30.0,59.4,Quelques nuages,9,85,6,30.1,NW
2,David Gaudu,2023-01-10,Bretagne,Finistère,145.87,5:01:45,1954,999,9999,999,99,29.0,78.6,Pluie,12,86,12,37.9,SW
3,David Gaudu,2023-01-09,Bretagne,Finistère,121.36,4:06:43,1012,999,9999,999,99,29.5,56.8,Pluie,8,74,3,46.3,WNW
4,David Gaudu,2023-01-04,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,75.60,2:43:34,1430,999,9999,999,99,27.7,60.7,Quelques nuages,12,85,12,6.6,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,David Gaudu,2021-12-09,Auvergne-Rhône-Alpes,Ain,114.68,4:00:23,1903,999,9999,999,99,28.6,75.3,Nuages,3,91,0,14.4,SW
382,David Gaudu,2021-12-06,Auvergne-Rhône-Alpes,Ain,71.54,2:30:02,804,999,9999,999,99,28.6,65.0,Pluie,6,91,2,18.5,SSW
383,David Gaudu,2021-12-05,Auvergne-Rhône-Alpes,Ain,36.64,1:15:50,242,999,9999,999,99,29.0,58.5,Quelques nuages,6,87,1,31.0,NW
384,David Gaudu,2021-12-02,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,113.26,4:01:31,1654,999,9999,999,99,28.1,72.7,Quelques nuages,8,74,7,8.4,NNW


In [88]:
df.to_csv("D:\Artus\École\A5\Webscraping\Code\Scraping_Gaudu_2023.csv")